This notebook aims to ingest the Hansard XML dumps, extract the information I care about, and write it to CSV files.

In [45]:
import os
import re
import sys
import multiprocessing

from bs4 import BeautifulSoup
import  pandas as pd

In [46]:
data_files = os.listdir('./data/')
data_files = [f for f in data_files if re.match('.*xml',f)]
data_files = ['./data/'+data_file for data_file in data_files]

In [47]:
class Speech:
    """Class to represent a speech in parliament"""
    
    def __init__(self,speech):
        """Should only be called from the constructor for Debate"""
        self.name = speech.find('name').get_text()
        self.text = speech.find('talk.text').get_text()
        self.party = speech.find('party').get_text()

class Division:
    
    def __init__(self,division,i):
        self.id = i
        self.result = division.find('division.result').get_text().strip()
        self.ayes = [name_tag.get_text() for name_tag in division.find('ayes').find_all('name')]
        try:
            self.noes = [name_tag.get_text() for name_tag in division.find('noes').find_all('name')]
        except:
            self.noes = []
        parent = division.parent
        grandparent = parent.parent
        self.parent_title = parent.title.get_text()
        self.grandparent_title = grandparent.title.get_text()
        
class Subdebate:
    
    def __init__(self,subdebate,i):
        self.id = i
        self.title = subdebate.find('title')
        self.speeches = [Speech(speech) for speech in subdebate.find_all('speech')]
        self.divisions = [Division(division,i) for i,division in enumerate(subdebate.find_all('division'))]

class Debate:
    """Class to represent a debate in parliament"""
    
    def __init__(self,debate,i):
        """Should only be called from the constructor for SittingDay"""
        self.id = i
        self.title = debate.find('title')
        self.subdebates = list()
        self.type = debate.type.get_text()
        subdebate = True
        i = 1
        while subdebate:
            subdebate=debate.find('subdebate.'+str(i))
            i+=1
            if subdebate:
                self.subdebates.append(Subdebate(subdebate,i-1))
        
        
        
class SittingDay:
    """Class to represent a whole parliamentary sitting day"""
    
    def __init__(self,day):
        """takes a beautifulsoup object"""
        self.date = day.find('date').get_text()
        debates = day.find_all('debate')
        self.debates = [Debate(debate,i) for i,debate in enumerate(debates)]
        self.chamber = day.find('chamber').get_text()
        if self.chamber == 'House of Reps':
            self.chamber = 'HoR'
        
    def write_divisions(self,path = '.'):
        name = list()
        vote = list()
        date = list()
        parent_title = list()
        grandparent_title = list()
        debate_type = list()
        debate_id = list()
        subdebate_id = list()
        chamber = list()
        division_id = list()
        for debate in self.debates:
            for subdebate in debate.subdebates:
                for division in subdebate.divisions:
                    all_names = division.ayes + division.noes
                    for member in all_names:
                        name.append(member)
                        if member in division.ayes:
                            vote.append('aye')
                        if member in division.noes:
                            vote.append('no')
                        date.append(self.date)
                        chamber.append(self.chamber)
                        parent_title.append(division.parent_title)
                        grandparent_title.append(division.grandparent_title)
                        debate_type.append(debate.type)
                        debate_id.append(debate.id)
                        subdebate_id.append(subdebate.id)
                        division_id.append(division.id)
                            
        data_frame = pd.DataFrame({'name':name,'vote':vote,'parent_title':parent_title,'grandparent_title':grandparent_title,
                                   'date':date,'chamber':chamber,'debate_type':debate_type,'debate_id':debate_id,'subdebate_id':subdebate_id,'division_id':division_id})
        data_frame.to_csv(path+'/'+self.date + '-' + self.chamber + '-' + 'divisions.csv',encoding = 'utf-8',index = False)
        
    def write_speeches(self,path='.'):
        """Write all the debates to a csv file (one file per day)"""
        #ToDo: extend to also include votes
        text = list()
        name = list()
        party = list()
        date = list()
        debate_title = list()
        subdebate_title = list()
        debate_type = list()
        chamber=list()
        debate_id = list()
        for debate in self.debates:
            for subdebate in debate.subdebates:
                for speech in subdebate.speeches:
                    text.append(speech.text)
                    name.append(speech.name)
                    party.append(speech.party)
                    date.append(self.date)
                    debate_title.append(debate.title)
                    subdebate_title.append(subdebate.title)
                    debate_type.append(debate.type)
                    chamber.append(self.chamber)
                    debate_id.append(debate.id)
        data_frame = pd.DataFrame({'name':name,'party':party,'date':date,'text':text,'chamber':chamber,'debate_title':debate_title,'subdebate_title':subdebate_title,'debate_type':debate_type,'debate_id':debate_id})
        data_frame.to_csv(path + '/' + self.date+ '-' + self.chamber +'-speeches.csv',encoding='utf-8',index=False)
        
    def write_proceedings(self,speeches_path='.',divisions_path='.'):
        self.write_speeches(speeches_path)
        self.write_divisions(divisions_path)

FixMe:
* ~~Split up debates into subdebates~~
* ~~Add divisions~~
* Add dictionary to associate MPs' names as listed in divisions with their names as listed elsewhere. 

Since building the BeautifulSoup representation of each XML file is somewhat slow, it's worth parallelising the process:

In [48]:
def xml_to_csv(x):
    data_file = x[0]
    speeches_path = x[1]
    divisions_path = x[2]
    with open(data_file,'r') as f:
        SittingDay(BeautifulSoup(f.read(),'lxml')).write_proceedings(speeches_path,divisions_path)
    return 'done'

(speeches_path,divisions_path) = ('../tidied_parliamentary_data/speeches','../tidied_parliamentary_data/divisions')

try:
    os.makedirs(speeches_path)
except OSError:
    pass #nothing to do if the directories already exist


try:
    os.makedirs(divisions_path)
except OSError:
    pass #nothing to do if the directories already exist

input = [(data_file,speeches_path,divisions_path) for data_file in data_files]

In [49]:
%%time
pool = multiprocessing.Pool(multiprocessing.cpu_count())
pool.map(xml_to_csv,input)

CPU times: user 158 ms, sys: 70.4 ms, total: 229 ms
Wall time: 4min 1s


In [51]:
with open(data_files[110],'r') as f:
    x = f.read()
divs = list()
for debate in SittingDay(BeautifulSoup(x)).debates:
    for subdebate in debate.subdebates:
        divs += subdebate.divisions
        
[div.noes for div in divs]

[['Bandt, AP',
  'Bird, SL',
  'Bowen, CE',
  'Burke, AS',
  'Butler, MC',
  'Butler, TM',
  'Byrne, AM',
  'Chalmers, JE',
  'Champion, ND',
  'Chesters, LM',
  'Clare, JD',
  'Claydon, SC',
  'Collins, JM',
  'Conroy, PM',
  'Dreyfus, MA',
  'Elliot, MJ',
  'Ellis, KM',
  'Ferguson, LDT',
  'Fitzgibbon, JA',
  'Giles, AJ',
  'Gray, G',
  'Hall, JG (teller)',
  'Hayes, CP',
  'Husic, EN',
  'Jones, SP',
  'King, CF',
  'Leigh, AK',
  'Macklin, JL',
  'MacTiernan, AJGC',
  'Marles, RD',
  'McGowan, C',
  'Mitchell, RG',
  'Neumann, SK',
  "O'Connor, BPJ",
  "O'Neil, CE",
  'Owens, J',
  'Parke, M',
  'Plibersek, TJ',
  'Ripoll, BF',
  'Rishworth, AL',
  'Rowland, MA',
  'Ryan, JC (teller)',
  'Snowdon, WE',
  'Swan, WM',
  'Thistlethwaite, MJ',
  'Thomson, KJ',
  'Vamvakinou, M',
  'Watts, TG',
  'Wilkie, AD',
  'Zappia, A'],
 ['Bandt, AP',
  'Bird, SL',
  'Bowen, CE',
  'Burke, AS',
  'Butler, MC',
  'Butler, TM',
  'Byrne, AM',
  'Chalmers, JE',
  'Champion, ND',
  'Chesters, LM',
  

In [ ]:
soup = BeautifulSoup(x)
soup.find_all('division')